# **Step 4 - The get menu tool**

Now that you have an understanding of function/tool calling, this next step should be easy. 🙂  

The goal of this step is let our agent handle requests about the available menu, which changes based on the day of the week. 

Let's start with the same initialization cell as before, which loads the enivronment variables, creates an AzureOpenAI client and the data access provider.   You've seen this code before so just run it and continue:

In [1]:
# Imports
import os
import json
import datetime
from openai import AzureOpenAI
from dotenv import load_dotenv
from datetime import datetime
from services.data_provider import DataProvider
from termcolor import cprint

# Environment setup
load_dotenv()
api_key=os.environ['AZURE_OPENAI_API_KEY']
api_version=os.environ['AZURE_OPENAI_API_VERSION']
deployment=os.environ['AZURE_OPENAI_DEPLOYMENT']

#Initialize AzureOpenAI client
client = AzureOpenAI(
  api_key=api_key,  
  api_version = api_version
  )

# This the data access layer
db:DataProvider=DataProvider()

messages = []

## Update the system message to handle the new requirement

Since we need to handle a new task, the `system_message` must be adapted to include this new requirement.  

Have a look at the new **Available menu** section and notice how we clearly added indications on how to react when a request for a menu comes in.

In [2]:
# Updated system message, we added other tasks to the role
system_message = f'''
Role: An assistant with expertise in handling questions about operative schedule and available menu of a food delivery service.
 
Instructions:
- Provide information about the available menu and the opening schedule.
- Kindly deny any request not regarding opening schedule and available menu.
- Always assume the date and time provided as input the reference date for all date calculations.
- If a date is not indicated use the reference date.
- If a time is not indicated assume it is 12 PM.
- Assume the following format: 'DD/MM/YYYY HH:MM' as the standard format for dates.

Steps:
  - Opening Schedule:
      1. Always use the calculated date based on reference date to check and return the opening schedule.
      2. Always indicate the desired data and time including weekday in the response.
      
  - Available menu:
      1. When the user asks for the a menu you should respond using the following template enclosed in triple quotes:

      ```
      Menu for <day-name> <requested date>
      
      Pasta
      
      1. <name> - <price> - <ingredients> - <label>
      2. <name> - <price> - <ingredients> - <label>
      3. ...
      4. ..      
      
      
      Pizza
      
      1. <name> - <price> - <ingredients> - <label>
      2. <name> - <price> - <ingredients> - <label>
      3. ...
      4. .. 
      
      Today's Special Pizzas
      
      1. <name> - <price> - <ingredients> - <label>
      2. <name> - <price> - <ingredients> - <label>
      3. ...
      4. ..   
      
      Drinks
      
      1. <name> - <price> 
      2. <name> - <price> 
      3. ...
      4. .. 
      
      Dessert
      
      1. <name> - <price> - <ingredients> - <label>
      2. <name> - <price> - <ingredients> - <label>
      3. ...
      4. .. 
      ```  
      
      2. If the request is about specific menu entries (e.g. 'do you have pasta', 'do you offer vegetarian food' or 'are there any specials?' ), together with the menu kindle answer the question.     
      3. If no specials are available for the day, do not include that section into menu.
      4. If no menu is available, kindly reply that there you can serve any food for that day..
      5. If the request relates to an item that is not available in the menu, kindly reply that it cannot be ordered.      
  
Expectation:
  - Provide a seamless experience to the user, by providing the requested information in a kind and timely manner, including all the necessary details and guiding the user to a possible follow up step.

Narrowing:  
  1. Deny all the requests referring to a date antecedent the reference date.
'''

## Introducting functions to look up the schedule & get the menu for a particular date:

Time for some code! 🙂

Here we have two functions:  

`operative_schedule`  
Is the same as before, nothing new for you.

`get_menu`  
Is the function in charge of returning the menu for a specific date. Here's an [example](services/data/menu/saturday.json) of the returned json.  

As before, don't care too much about the code itself, what is important is to know is that there is a function in charge of returning the data when invoked.

**Note** In order to make the part of code that invokes the functions more flexible, the 2 functions are added to an `available_functions` dictionary that we'll use later.

In [3]:

# Returns the operative schedule on provided date for the user_id provided (if indicated, it might have a special treatment)
def operative_schedule(date:str, user_id:str=None):
    """
    Retrieves and validates the operating schedule for a given date and user.

    This function determines the operative schedule for a specified date. It first extracts the weekday from the provided date. 
    Then, it checks if the user is a special client and retrieves the schedule from the database accordingly. 
    If the schedule status is "open," the function verifies whether the requested date/time falls within the schedule's operational hours.
    If the time is outside the operational hours, it updates the schedule status to "closed."

    Parameters:
    ----------
    date : str
        The date and time in the format "dd/mm/YYYY HH:MM" for which to check the schedule.
    user_id : str, optional
        The unique identifier of the user. This is used to determine if the user is a special client. Default is None.

    Returns:
    -------
    str
        A JSON string representing the schedule model with updated status, if applicable.

    Notes:
    -----
    - The function relies on a database (`db`) with methods `is_special_client(user_id)` and `get_schedule(day, is_special)`.
    - The `get_schedule` method returns a schedule object with properties `status`, `start`, `end`, and `model_dump_json()`.
    - The function assumes the provided `date` string is in the correct format and valid.
    
    Example:
    -------
    >>> operative_schedule("23/06/2024 14:30", "user123")
    '{"status": "open", "start": "09:00", "end": "17:00"}'
    """
    weekday= datetime.strptime(date, "%d/%m/%Y %H:%M").strftime("%A")    
    
    # Check if the user is a special client
    is_special = db.is_special_client(user_id)
    # Get the operative schedule from the database
    schedule=db.get_schedule(day=weekday, is_special=is_special) 
    # Check if the requested date/time is within the working schedule, if not we update the status to closed       
    if schedule.status == "open":        
        time= datetime.strptime(date, "%d/%m/%Y %H:%M").time()
        opening = datetime.strptime(schedule.start, "%H:%M").time()
        closing = datetime.strptime(schedule.end, "%H:%M").time()        
        if time < opening or time > closing:
            schedule.status = "closed"            
        
    return schedule.model_dump_json()


# Returns the menu on the provided date    
def get_menu(date:str):
    target_date= datetime.strptime(date, "%d/%m/%Y %H:%M") 
    weekday= target_date.strftime("%A")
    
    # Get the menu from the database
    menu= db.get_menu(day=weekday)
    #return the response as JSON string    
    return menu.model_dump_json()


# We create a dictionary with the available functions, using function name as key
available_functions = {
            "operative_schedule": operative_schedule,
            "get_menu": get_menu
    }

In [4]:
operative_schedule("23/06/2024 00:00")

'{"day":"Sunday","start":"9:00","end":"21:00","status":"closed"}'

Here's the updated tools object, we added the definitions of the new `get_menu` function so that the model knows of its existence.

In [5]:
# Define the function available to the LLM
tools = [
        {        
            "type": "function",    
            "function": {
                "name": "operative_schedule",
                "description": "Useful when you need to know the operative schedule on a specific date.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "date": {
                            "type": "string",
                            "description": "The date for which you want to know the schedule in format 'DD/MM/YYYY HH:MM' (e.g. '31/12/2023 15:00').",
                        },
                        "user_id": {
                            "type": "string",
                            "description": "The optional user id in form '#<4-digits-number>'.",
                        }                        
                    },
                    "required": ["date"]
                }
            },            
        },
               {        
            "type": "function",    
            "function": {
                "name": "get_menu",
                "description": "Useful when you need to know the menu for a specific date.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "date": {
                            "type": "string",
                            "description": "The date for which you want to know the available menu in format 'DD/MM/YYYY HH:MM' (e.g. '29/05/2024 17:00').",
                        }                  
                    },
                    "required": ["date"]
                }
            },            
        }
    ]

Nothing new here, is the same function that invokes the LLM you've seen in the previous step.

In [6]:
# Invoke the LLM with the provided input, if none provided it just uses the existing messages
def invoke_llm(input:str=None)->str:
    if input:
          messages.append({'role': 'user', 'content': input})
    response = client.chat.completions.create(
    model=deployment,    
    messages = messages,
    tools=tools,
    tool_choice="auto",
    temperature=0.3)

    return response.choices[0].message

And here's the part that handles the conversation between the LLM and the `operative_schedule` and `get_menu` tools.  
The code has been refactored to handle any variable number of tools invocation requests and dynamically invoke the requested function.  

The behavior remains the same as before, but we encourage you to wear your engineer hat ⛑️ and play/debug/test and change the `user_message` to fully understand the whole conversation flow.

In [7]:
# This is the user request, change it as you wish and re-run this cell.😉
user_message = "Are you open tomorrow at noon, if so, what can i have for lunch?"

messages = [{'role': 'system', 'content': system_message}]

# We add current date and time to the prompt to let LLM know the reference date and time to use
now = datetime.now().strftime("%A, %d/%m/%Y %H:%M")
input_request = f"Given that now is {now}, handle the following user request enclosed in triple backticks: ```{user_message}```" 
cprint(f"Input: {input_request}", "blue") 

while True:
    # We invoke the LLM with the input request
    response = invoke_llm(input=input_request)    
    # We check if the response contains tool calls
    if response.tool_calls:
        cprint(response, "cyan")
        # Note that we add the request from the LLM to the messages list
        messages.append(response)
        for tool_call in response.tool_calls:
            # We get the function to call from the available_functions dictionary
            function_to_call = available_functions[tool_call.function.name]
            # We extract the arguments from the tool call            
            function_args = json.loads(tool_call.function.arguments)
            # We call the function with the extracted arguments
            cprint (f"Invoking function: {tool_call.function.name}...", "green")
            function_response = function_to_call(**function_args)
            cprint (f"Function call completed...", "yellow")
            # We append the function response to the messages list, note that we add the tool_call id to the response to let the LLM identify it
            messages.append(
                {
                    "tool_call_id": tool_call.id,
                    "role": "tool", 
                    "name": tool_call.function.name,
                    "content": function_response,
                }) 
        # We set input request to None to let the LLM know that it should continue with the existing conversation that contains the function responses
        input_request = None
        cprint (f"Control back to the LLM...", "magenta")
    else:
        # If the response does not contain tool calls, it means LLM has completed the required tasks
        cprint (response.content, "yellow")
        break    

Input: Given that now is Tuesday, 30/07/2024 11:45, handle the following user request enclosed in triple backticks: ```Are you open tomorrow at noon, if so, what can i have for lunch?```
ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_KsErrTpHwt2wBu0lIDDv87nV', function=Function(arguments='{"date": "31/07/2024 12:00"}', name='operative_schedule'), type='function'), ChatCompletionMessageToolCall(id='call_SzACM2w0rIZXuw0jNzZ9eOWM', function=Function(arguments='{"date": "31/07/2024 12:00"}', name='get_menu'), type='function')])
Invoking function: operative_schedule...
Function call completed...
Invoking function: get_menu...
Function call completed...
Control back to the LLM...
Yes, we are open tomorrow, Wednesday, 31/07/2024 at noon.

Here is the menu for Wednesday, 31/07/2024:

```
Menu for Wednesday 31/07/2024

Pasta

1. Spaghetti Carbonara - 12.99 - spaghetti, bacon, eggs, parmesan cheese - 
2. Spaghetti Bolo

Did you try some of this question?  

1- *"Are you open today?"*  
2- *"What is the menu for today?"*

Did you notice that, thanks to the "reasoning" of the LLM, question 1 invoked only `operative_schedule` function while the second invoked `get_menu`?  

What if you ask: *"Are you open tomorrow at noon?, if so, what can i have for lunch?"*

 (highlight that only get_menu function is requested.)  
3-Show that using "Are you open the day after tomorrow at 6 AM?" will result in the opening_schedule function called. So the LLM can pick the proper one.  
4-Show that asking: "Are you open tomorrow at noon, if so, what pasta do you serve?" turn into the request to execute both functions.  

Are you start getting the potential of combining an LLM with one or more tools?


### **Challenge Time again!** 😎

Until now we ingested the current datetime in the prompt:

```python
now = datetime.now().strftime("%A, %d/%m/%Y %H:%M")
input_request = f"Given that now is {now}, handle the following user request enclosed in triple backticks: ```{user_message}```" 
```

Can you think of a way to avoid it?

See [Notebook 05](05_get_current_date_tool.ipynb) for a solution.